In [ ]:
import numpy as np
import pylab as plt
import svgwrite
from skimage import io, filters, measure, feature, exposure, morphology
from scipy import ndimage, spatial
from scipy.ndimage import rotate
import os
import random
from IPython.display import SVG
%matplotlib inline

In [ ]:
ao_monkey = io.imread('TestMonkey/AO.png')
uv_monkey = io.imread('TestMonkey/UV.png')
normal_monkey = io.imread('TestMonkey/Normal.png')
original_monkey = io.imread('TestMonkey/Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0])
coords = feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=5)
print(coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],"ro")
plt.imshow(edge_monkey)

In [ ]:
def center_angle(p1, p2):
    x_difference = p2[1]-p1[1]
    y_difference = p2[0]-p1[0]
    return np.rad2deg(np.arctan2(y_difference, x_difference))

In [ ]:
mean_x = np.mean(coords[1,:])
mean_y = np.mean(coords[0,:])
mean_point = [mean_y, mean_x]

start = coords[np.where(spatial.distance.cdist([[0, 0]], coords).max())]
angles = []
for i in coords:
    angles.append(center_angle(i, mean_point))

angles = np.asarray(angles)
coords = coords[angles.argsort()]
print(start, coords)
plt.figure()
plt.plot(coords[:,1],coords[:,0],)
plt.imshow(edge_monkey)

In [ ]:
dwg = svgwrite.Drawing('stroke.svg')

dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

for i in range(0,10):
    genid = chr(i+48)
    gradline = dwg.linearGradient((0, 0), (1, 0), id=genid)
    gradline.add_stop_color(1, 'white', 0)
    gradline.add_stop_color(round(random.uniform(0.85, 0.95), 2), 'black')

    dwg.defs.add(gradline)
    
    start_x = random.randint(10, 100)
    start_y = random.randint(10, 100)
    
    dwg.add(dwg.line((start_x, start_y),
                     (start_x + random.randint(25, 100), start_y + random.randint(25, 100)),
                     stroke="url(#" + genid + ")",
                     stroke_linecap="round"))

display(SVG(dwg.tostring().replace('><','>\n<')))

dwg.save()

In [ ]:
def sketch(points, name):
    precision = 5
    dwg = svgwrite.Drawing(name)
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))
    
    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    #gradline.add_stop_color(1, 'white', 0.1)
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    for i in range(1, len(points)):
        dwg.add(dwg.line((round(points[i-1][0], precision),(round(points[i-1][1], precision))),
                         ((round(points[i][0], precision), (round(points[i][1], precision)))),
                         stroke="url(#grad)",
                         stroke_linecap="round"))

    display(SVG(dwg.tostring().replace('><','>\n<')))

    return dwg

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0] + normal_monkey[:,:,0])
new_edge_monkey =np.flip(np.rot90(edge_monkey), axis=0)
contours = measure.find_contours(new_edge_monkey, 0.17, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
plt.figure(figsize=(6,6))
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=2, color="black")

In [ ]:
def path_sketches(points):
    precision = 5
    dwg = svgwrite.Drawing('path1.svg')

    gradline = dwg.linearGradient((0, 0), (1, 0), id="grad")
    gradline.add_stop_color(1, 'white', 0.8)
    gradline.add_stop_color(0.9, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0][0,0], points[0][0,1]), fill='black', stroke='black')
    for pointSet in points:
        path.push('M', pointSet[0,0], pointSet[0,1])
        for p in pointSet[1:]:
            path.push(('L', p[0], p[1]))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketches(simplified_contours)
path_out.save()

In [ ]:
def angle(p1):
    ang = np.arctan2(p1[0], p1[1])
    return np.rad2deg((ang) % (2 * np.pi))

In [ ]:
def path_sketch(points, corners=[]):
    corners[:,[0,1]] = corners[:,[1,0]]
    dwg = svgwrite.Drawing('path1.svg')
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')
    
    dwg.defs.add(gradline)
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black')
    for i in range(1, len(points)):
        path.push(('L', points[i,0], points[i,1]))
        print(points[i-1], points[i])
        if points[i] in corners:
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i-1,0])*2.5, points[i,1]+(points[i,1]-points[i-1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
            
            dwg.add(dwg.line((points[i,0],points[i,1]),
                             (points[i,0]+(points[i,0]-points[i+1,0])*2.5, points[i,1]+(points[i,1]-points[i+1,1])*2.5),
                             stroke='black',
                             stroke_width=round(random.uniform(0.6, 0.85), 2),
                             stroke_linecap='round'))
    
    dwg.add(path)
    
    return dwg

In [ ]:
path_out = path_sketch(simplified_contours[0], feature.corner_peaks(feature.corner_fast(edge_monkey), min_distance=20))
path_out.save()

In [ ]:
edge_monkey = filters.sobel(normal_monkey[:,:,0]) + filters.sobel(ao_monkey[:,:,0])
feature_monkey = filters.sobel(normal_monkey[:,:,0]) - filters.sobel(ao_monkey[:,:,0])
bin_edge_monkey = np.where(edge_monkey<0.1, 0, 1)
bin_feature_monkey = np.where(feature_monkey<0.1, 0 ,1)
plt.figure(figsize=(20,20))
plt.imshow(ndimage.gaussian_filter(bin_feature_monkey, sigma=0.12), cmap = plt.cm.Greys)

In [ ]:
def momentum_sketch_fake(dwg, points):
    velocity = 0.1
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = 0.1
        currentPos = np.copy(points[i-1])
        
        angle = np.rad2deg(np.arctan2(points[i][1], points[i][0]))
        while lessThan(points[i-1], currentPos, points[i]):
            currentPos = [currentPos[0]+( velocity*np.cos(angle) ), currentPos[1]+( velocity*np.sin(angle) )]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + 0.2
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def momentum_sketch(dwg, points):
    velocity_base = 0.1
    acceleration = 0.4
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = velocity_base
        currentPos = np.copy(points[i-1])
        
        while lessThan(points[i-1], currentPos, points[i]):
            difference_vector = points[i]-points[i-1]
            currentPos = [currentPos[0] + (difference_vector[0]*velocity), currentPos[1] + (difference_vector[1]*velocity)]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + acceleration
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def normal_sketch(dwg, points):
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke='black', stroke_linecap='round')
    for i in range(1, len(points)):
        currentPos = np.copy(points[i-1])
        
        path.push(('L', currentPos[0], currentPos[1]))
        
    dwg.add(path)
        
    return dwg

def momentum_sketch_test(dwg, points):
    velocity_base = 0.1
    acceleration = 0.4
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]), stroke_linecap='round')
    for i in range(1, len(points)):
        velocity = velocity_base
        currentPos = np.copy(points[i-1])
        
        while lessThan(points[i-1], currentPos, points[i]):
            difference_vector = points[i]-points[i-1]
            currentPos = [currentPos[0] + (difference_vector[0]*velocity), currentPos[1] + (difference_vector[1]*velocity)]
            path.push(('L', currentPos[0], currentPos[1]))
            velocity = velocity + acceleration
            
        path.push(('M', points[i][0], points[i][1]))
        
    dwg.add(path)
        
    return dwg

def guided_sketch(dwg, points):
    velocity_base = 0.1
    acceleration = 0.4
    turn_rate = 2
    
    path = dwg.path(('M', points[0,0], points[0,1]), fill='none', stroke="black", stroke_linecap='round')
    for i in range(1, len(points)):
        currentPos = np.copy(points[i-1])
        difference_vector = points[i]-points[i-1]
        
        if i < len(points)-1:
            next_difference_vector = points[i+1]-points[i]
        else:
            next_difference_vector = difference_vector
            
        angle = dotProductAngle(difference_vector, next_difference_vector)
        for j in range(1, turn_rate):
            nextPos = [(currentPos[0] + (difference_vector[0]*(j/turn_rate)) * np.cos(angle)), currentPos[1] + (difference_vector[1]*(j/turn_rate) * np.sin(angle))] 
            path.push(('L', nextPos[0], nextPos[1]))
            
    dwg.add(path)
        
    return dwg

def lessThan(startPos, currentPos, endPos):
    return np.linalg.norm(currentPos-startPos) < np.linalg.norm(endPos-startPos)

def dotProductAngle(v1, v2):
    #re-arranged dot product for theta
    theta = np.arccos( np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)) )
    if np.isnan(theta):
        return 0
    else:
        return np.arccos( np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)) )

def rotMatrix(theta):
    return [[np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]]

def initCanvas(name):
    dwg = svgwrite.Drawing(name)
    
    dwg.add(dwg.rect(insert=(0, 0), size=('100%', '100%'), rx=None, ry=None, fill='white'))

    gradline = dwg.linearGradient((0, 0), (0, 0), id="grad")
    gradline.add_stop_color(0.1, 'white')
    gradline.add_stop_color(1, 'black')

    dwg.defs.add(gradline)
    
    return dwg

### Walking the image approximately

* Needs to take an image
* Return the walked coords
* Reduce the number of points to walk from raw
* Capture the features of the image
* Single walk around the edge
* Encapsulate the whole image

In [ ]:
def getUnwalkedCoords(image):
    coords = np.where(image == 1)
    zipped_coords = np.dstack(coords)
    return zipped_coords.reshape(zipped_coords.shape[1], zipped_coords.shape[2])

def getNeighbours(currentPixel, centrePoint):
    neighbours = np.array([[currentPixel[0]+1, currentPixel[1]+1],
            [currentPixel[0]+1, currentPixel[1]],
            [currentPixel[0]+1, currentPixel[1]-1],
            [currentPixel[0], currentPixel[1]+1],
            [currentPixel[0], currentPixel[1]-1],
            [currentPixel[0]-1, currentPixel[1]+1],
            [currentPixel[0]-1, currentPixel[1]],
            [currentPixel[0]-1, currentPixel[1]-1],
           ])
    
    neighbours_dist = map(lambda x: np.linalg.norm(x-centrePoint), neighbours)
    neighbours = neighbours[np.argsort(np.array(list(neighbours_dist)))]
    return neighbours
    
def nextNeighbour(currentPixel, thinned_image, centrePoint):
    neighbours = getNeighbours(currentPixel, centrePoint)
    for n in neighbours:
        n = np.array(n).astype(int)
        if thinned_image[n[0], n[1]] == 1:
            return n
    return np.array([-1,-1])

def walkNeighbourhood(image):
    thinned_image = morphology.thin(np.where(image<0.1, 0, 1))
    thinned_image = np.where(thinned_image<0.1, 0, 1)
    centre = getCentre(thinned_image)
    
    totalPixels = exposure.histogram(thinned_image, nbins=2)[0][1]
    currentPixel = np.array([-1, -1])
    walkingOrder = []
    index=-1
    while totalPixels > 0:
        
        if np.array_equal(currentPixel, np.array([-1, -1])):
            #print(len(getUnwalkedCoords(thinned_image)))
            currentPixel = getUnwalkedCoords(thinned_image)[0]
            walkingOrder.append([])
            index += 1
        else:
            #print(currentPixel, totalPixels)
            thinned_image[currentPixel[0], currentPixel[1]] = 0
            walkingOrder[index].append(currentPixel)
            currentPixel = nextNeighbour(currentPixel, thinned_image, centre)
            totalPixels -= 1
            
    return walkingOrder

#idea from https://www.geeksforgeeks.org/program-check-three-points-collinear/
def checkColinear(p1, p2, p3):
    return (p1[0] * (p2[1]-p3[1]) + p2[0] * (p1[1]-p3[1]) + p3[0] * (p1[1]-p2[1])) == 0

def getCentre(image):
    return np.mean(np.where(image==1), axis=1)

#Checking colinearity should probably be used here (maybe a recursive solution??)
def findMajorAnchors(pointsList):
    for feature in pointsList:
        changed = True
        #if the point set wasn't reduced then go to the next point set
        while changed:
            changed = False
            i = 0
            while True:

                if len(feature) < 3:
                    changed = False
                    break
                    
                if len(feature)-2 <= i:
                    break
                    
                #print(feature[i],feature[i+1],feature[i+2])   
                if checkColinear(feature[i],feature[i+1],feature[i+2]):
                    feature.pop(i+1)
                    changed = True
                    
                i += 1
    return pointsList

pointToDraw = walkNeighbourhood(np.flip(np.rot90(edge_monkey), axis=0))
canvas = initCanvas("pathV0.svg")
pointToDraw = findMajorAnchors(pointToDraw)
for feature in pointToDraw:
    canvas = guided_sketch(canvas, np.array(feature))
outline =  findMajorAnchors(walkNeighbourhood(np.flip(np.rot90(filters.sobel(ao_monkey[:,:,0])), axis=0)))
canvas = guided_sketch(canvas, np.array(outline[0]))
    
canvas.save()

In [ ]:
path_out = initCanvas('momentum.svg')
path_out = momentum_sketch(path_out, simplified_contours[0])
path_out.save()

In [ ]:
feature_monkey = np.flip(np.rot90(filters.sobel(normal_monkey[:,:,0]) - filters.sobel(ao_monkey[:,:,0])), axis=0)
bin_feature_monkey = np.where(feature_monkey<0.1, 0 ,1)
contours = measure.find_contours(bin_feature_monkey, 0.17, fully_connected="high")
feature_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
plt.figure(figsize=(6,6))
for n, contour in enumerate(feature_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=2, color="black")

In [ ]:
path_out = initCanvas('fullMomentum.svg')
path_out = momentum_sketch(path_out, simplified_contours[0])
for i in range(0, len(feature_contours)):
    path_out = momentum_sketch(path_out, feature_contours[i])
path_out.save()